In [1]:
import twitter

In [2]:
with open('../credential/bearer_token.txt') as f:
    BEARER_TOKEN = f.read()

t_h = twitter.Twitter(auth=twitter.OAuth2(bearer_token=BEARER_TOKEN))

In [3]:
import pandas as pd
jak_trend = pd.read_csv('../dataset/trend.txt')
jak_trend.head(10)

,Unnamed: 0,name,url,promoted_content,query,tweet_volume
0,0,xiao,http://twitter.com/search?q=xiao,NaN,xiao,51789.0
1,1,#AMAsTNT,http://twitter.com/search?q=%23AMAsTNT,NaN,%23AMAsTNT,8607535.0
2,2,#BTSxAMAs,http://twitter.com/search?q=%23BTSxAMAs,NaN,%23BTSxAMAs,5166619.0
3,3,#TelkomDukungUMKMmandalika,http://twitter.com/search?q=%23TelkomDukungUMK...,NaN,%23TelkomDukungUMKMmandalika,NaN
4,4,#IVE_1st_ConceptPhoto,http://twitter.com/search?q=%23IVE_1st_Concept...,NaN,%23IVE_1st_ConceptPhoto,11268.0
5,5,#IkatanCintaEp520,http://twitter.com/search?q=%23IkatanCintaEp520,NaN,%23IkatanCintaEp520,NaN
6,6,Nessie,http://twitter.com/search?q=Nessie,NaN,Nessie,20606.0
7,7,Edi Witjara,http://twitter.com/search?q=%22Edi+Witjara%22,NaN,%22Edi+Witjara%22,NaN
8,8,Mas Gilang,http://twitter.com/search?q=%22Mas+Gilang%22,NaN,%22Mas+Gilang%22,NaN
9,9,Ariel,http://twitter.com/search?q=Ariel,NaN,Ariel,19763.0


In [8]:
jak_trend.get('name').values

array(['xiao', '#AMAsTNT', '#BTSxAMAs', '#TelkomDukungUMKMmandalika',
       '#IVE_1st_ConceptPhoto', '#IkatanCintaEp520', 'Nessie',
       'Edi Witjara', 'Mas Gilang', 'Ariel', 'MEW MEW SO HOT', 'Ngeluh',
       'ITDC Group', 'Shenhe', 'MS Glow', 'SPBU', 'Arsyad', 'Kojic',
       'YESEO FOR DAZED', 'Wine', 'Healing', 'Juragan', 'Marriage',
       'Arteria', 'Bram', 'Sapu', 'Yunjin', 'Penjilat', 'Polearm',
       'Provinsi Jawa', 'Selingkuh', 'Sejarah', 'Karma', 'Emil Salim',
       'Operasi', 'SF9 TRAUMA OUT NOW', 'Jaksel', 'Dp 150k', 'Ganyu',
       'Eula', 'EMS TAX', 'ayato', 'OPEN PO', 'Itto', 'Eunkwang',
       'TBL TBL TBL', 'GO Line', '#peShan', '#Seungmin', '#HARUKYU'],
      dtype=object)

In [25]:
import re
import numpy as np

for trend in jak_trend.get('name').values:
    print('Starting', trend, end='... ')
    tweet_df = pd.DataFrame()
    max_tweets = 300
    since_id_buff = ''

    # Loop. Expect the amount of tweets fetched to exceed the value of max_tweets a bit. Each of the following tweets will be unique.
    while len(tweet_df) < max_tweets:
        query = t_h.search.tweets(q=trend, lang='en', count=100, tweet_mode='extended') if since_id_buff == '' else t_h.search.tweets(q=trend, lang='en', count=100, tweet_mode='extended', max_id=since_id_buff)
        since_id_buff = re.search(r'max_id=(.+)&q', query['search_metadata']['next_results']).group(1)
        query = pd.DataFrame(query['statuses'])

        # Get full text, put it in a new header
        query_rt = []
        for i in query['retweeted_status']:
            try:
                query_rt.append(i['full_text'])
            except:
                query_rt.append(np.nan)
        # print(query_rt)
        query_text = []
        for i in range(len(query_rt)):
            # print(i, end='\r')
            query_text.append(query_rt[i] if not pd.isnull(query_rt[i]) else query['full_text'].loc[i])

        query['inferred_text'] = query_text

        # Fixing
        tweet_df = tweet_df.append(query).reset_index().drop('index', axis=1) if tweet_df is not None else query

        if len(tweet_df) >= max_tweets:
            tweet_df = tweet_df.drop_duplicates(subset=['inferred_text']).reset_index().drop('index', axis=1)

    tweet_df.to_json('../dataset/trends/'+trend+'.json', orient='index')
    print('Done!')

Starting xiao... Done!
Starting #AMAsTNT... Done!
Starting #BTSxAMAs... Done!
Starting #TelkomDukungUMKMmandalika... 

KeyError: 'retweeted_status'

In [19]:
# Check the last tweet dataframe
tweet_df.shape

(314, 32)

In [20]:
tweet_df.columns

Index(['created_at', 'id', 'id_str', 'full_text', 'truncated',
       'display_text_range', 'entities', 'metadata', 'source',
       'in_reply_to_status_id', 'in_reply_to_status_id_str',
       'in_reply_to_user_id', 'in_reply_to_user_id_str',
       'in_reply_to_screen_name', 'user', 'geo', 'coordinates', 'place',
       'contributors', 'retweeted_status', 'is_quote_status', 'retweet_count',
       'favorite_count', 'favorited', 'retweeted', 'lang', 'extended_entities',
       'possibly_sensitive', 'quoted_status_id', 'quoted_status_id_str',
       'quoted_status', 'inferred_text'],
      dtype='object')

In [24]:
tweet_df.to_json('../dataset/trends/nyoba.json', orient='index')